In [1]:
!pip install pyFAI --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 MB 14.5 MB/s eta 0:00:00


In [ ]:
#@title Imports
#!pip install /content/drive/MyDrive/'Colab Notebooks'/packages
!pip install pystackreg -q


#from image_processing.image_processing import ImageProcessing, ImageAnalysis
from skimage.feature import match_template
from google.colab import drive
from google.colab.patches import cv2_imshow
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.ext.watershed import InverseWatershed
from pyFAI.gui import jupyter
import pyFAI
import tifffile
from google.colab import drive
drive.mount('/content/drive')
%matplotlib inline
seed = 42

## **1.Implementação do processo de geração do I(Q) automatizado**

In [3]:
#@title Funções gerais
"""image_processing.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1cztFQa3vJ-XnFKiXgGi3mrgvNOCEPl3k
"""

import os
import numpy as np
from PIL import Image
import cv2
from pystackreg import StackReg
import pandas as pd
import math
import matplotlib.pyplot as plt
import pyFAI
import copy

class ImageProcessing:
    def __init__(self, path):
        self.path = path

    def load_images(self, Binary = 1):
        if ~os.path.isdir(path):
          img = Image.open(self.path)
          img = np.array(img)
          return img
        else:
          images_list = os.listdir(self.path)
          images_names = [image for image in images_list if image.lower().endswith(".tif")]
          images_names.sort()
          images = []
          images_array = []
          for filename in images_names:
              img = Image.open(os.path.join(self.path, filename))
              images.append(img)
              if Binary == 0:
                img[img == 255] = 1
                img= img[:,:,0]
              img = np.array(img)
              images_array.append(img)
          images_array = np.array(images_array)
          return images_array, images_names

    def load_mask(self, mask_path):
      return np.array(Image.open(mask_path))

#Função teste para inverter as imagens a partir do centro
    def invert_images(self, images, center, beamstopper = "right"):
      for i in range(len(images)):
        if beamstopper == "left":
          images[i] = images[i][:,:-1]
        left_side = images[i][:,:center]
        mirrored_half = np.flip(left_side, axis=1)
        mirrored_image = np.concatenate((left_side, mirrored_half), axis=1)
        mirrored_image = mirrored_image[:,106:4202]
        cv2.imwrite(path + '/Sem beamstopper/' + names[i][:-4] + '.jpg', mirrored_image)
      return None

    #Função para alinhar as imagens e somá-las
    def stack_translate(self, stack):
        out_previous = StackReg(StackReg.TRANSLATION).register_transform_stack(stack, reference='previous')
        return sum(out_previous)

    def stack_rotate(self, stack):
        out_previous = StackReg(StackReg.RIGID_BODY).register_transform_stack(stack, reference='previous')
        return sum(out_previous)

    def subtract_mask(self, image, mask):
      if image.shape == mask.shape:
        image[mask==255] = 0
        return image
      else:
        raise Exception('A imagem tem'f'{image.shape} e a mascara tem 'f'{mask.shape}')


#Retirar os defeitos fixos das imagens
    def fixed_defects_mask(self, image, microscope):
      if microscope.lower() in ("titan"):
        image[2140:2160, 2030:2070]=False
        image[:,4087:]=False
        image[:,0:5]=False
        image[4051:4053]=False
        image[3072:]=False
        return image
      else:
        image[:,:7]=False
        image[3072:]=False
        image[2136,1976:2225]=False
        return image

    def save_iq(self, iq, name):
        if os.path.isfile(self.path):
    # Get the directory part of the file path
          directory = os.path.dirname(self.path)
          iq = pd.DataFrame(np.transpose(np.array(iq)))
          iq.to_csv(f'{directory}/'f'{name}.csv', sep='\t')
          return None
        else:
          iq = pd.DataFrame(np.transpose(np.array(iq)))
          iq.to_csv(f'{self.path}/'f'{name}.csv', sep='\t')
          return None

    def save_iq_only_y(self, iq, name):
        if os.path.isfile(self.path):
    # Get the directory part of the file path
          directory = os.path.dirname(self.path)
          iq = pd.DataFrame(np.transpose(np.array(iq)))
          iq.drop(columns=[0], inplace=True)
          iq.to_csv(f'{directory}/'f'{name}.csv', sep='\t', index=False, header=False)
          return None
        else:
          iq = pd.DataFrame(np.transpose(np.array(iq)))
          iq.drop(columns=[0], inplace=True)
          iq.to_csv(f'{self.path}/'f'{name}.csv', sep='\t', index=False, header=False)
          return None

#Encontrar o centro com a transformada de Hough para usar como chute inicial
class ImageAnalysis:
    def __init__(self):
        pass

    def find_center(self, image,  r, R):
        if image is None:
            return "Image not loaded properly. Check the image path."

        blur = cv2.GaussianBlur(masked_stacked, (5, 5), 0)
        final_im = cv2.normalize(src=blur, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        circles = cv2.HoughCircles(final_im, cv2.HOUGH_GRADIENT, dp=1, minDist=100, param1=50, param2=30, minRadius= 10, maxRadius=5000)

        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            center_x, center_y, r = circles[0]
            return center_x, center_y, r
        else:
            return "No circles detected in the image."

    def fine_tune_center(self, image, center_x, center_y):
        radius = 1
        min_eqm = float('inf')
        best_center_x = center_x
        best_center_y = center_y
        azimuth = {"direita":[120, 140], "esquerda":[300,320], "cima":[80,100], "baixo":[260, 280]}
        profiles = []
        ai = pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=0.1, pixel1=1e-4, pixel2=1e-4)

        for dx in range(-radius, radius+1):
            for dy in range(-radius, radius+1):
                ai.setFit2D(2048, center_x, center_y)

                for angle in azimuth:
                    res = ai.integrate1d(image, 3000, unit='2th_deg', azimuth_range=(azimuth[angle]), method="ocl_lut_integr")
                    np.array(res)
                    profiles.append(res)

                eqm = self.MSE(profiles[0][0][100:500], profiles[1][1][100:500]) + self.MSE(profiles[2][1][100:500], profiles[3][1][100:500])

                if eqm < min_eqm:
                    min_eqm = eqm
                    best_center_x = center_x
                    best_center_y = center_y

                center_x = center_x + dx
                center_y = center_y + dy

        return best_center_x, best_center_y

    def plot_iq(self, x):
        x = np.array(x)
        fig, ax = plt.subplots(1, 1, figsize=(10, 7))
        fig.suptitle("Integrated Iq", fontsize=16)
        ax.set_xlim(0, x[0].max())
        ax.set_ylim(0, x[1].max())
        ax.set_yticks(np.arange(0, 100, 160))
        ax.grid()
        ax.grid(which='minor', linestyle='--')
        ax.minorticks_on()
        ax.plot(x[0], x[1], label='Python')
        plt.show()
        return fig, ax

    def MSE(self, x, y):
        return math.sqrt(1/len(x) * sum((x - y)**2))

    def azimuthal_projection(shape, center, original_image):
        output_image = np.zeros_like(shape)
        x_values, y_values = shape
        for x in range(x_values):
          for y in range(y_values):
            r = np.sqrt((x-center[0])**2 + (y-center[1])**2)

            theta = np.arctan2(y - center[0], x - center[1])

            src_x = int(center_x + r * np.cos(theta))
            src_y = int(center_y + r * np.sin(theta))


        if 0 <= src_x < original_image.shape[0] and 0 <= src_y < original_image.shape[1]:
            output_image[y, x] = original_image[src_y, src_x]

        return output_image

In [ ]:
#@title 1.1 Carregando as imagens

file_path = '/content/drive/MyDrive/CNPEM/Jacutingaita/Analises/PDF/'
path = "/content/drive/MyDrive/CNPEM/Jacutingaita/Analises/PDF/Corning/CAS40/B4/Corning_CAS40_ROI1_0.0063s_20_B4-Stacked.tif"

#path = "/content/drive/MyDrive/CNPEM/Jacutingaita/Analises/DPC/Esferas/ROI-3"

process = ImageProcessing(path)
x = process.load_images()

In [ ]:
#@title 1.2. Retirando os defeitos

#import general_functions

mask_path = "/content/drive/MyDrive/CNPEM/Jacutingaita/Analises/PDF/MASKS/mask_titan.tif"

analysis = ImageAnalysis()

mask = process.load_mask(mask_path)
masked_stacked = copy.deepcopy(x.reshape(1024,1024))
#masked_stacked = process.subtract_mask(masked_stacked, mask)
masked_stacked = process.fixed_defects_mask(masked_stacked,'titan')
plt.imshow(masked_stacked)

In [ ]:
#@title 1.3. Encontrar o centro
from matplotlib.patches import Circle

blur = cv2.GaussianBlur(x, (5, 5), 30)
final_im = cv2.normalize(src=blur, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
#final_im = np.where(final_im > 100, 255, final_im)
circles = cv2.HoughCircles(final_im, cv2.HOUGH_GRADIENT,
                           dp=1,
                           minDist=100,
                           param1=50,
                           param2=30,
                           minRadius= 10,
                           maxRadius=5000)


circles = np.round(circles[0, :]).astype("int")
center_x, center_y, r = circles[0]

circle1 = Circle((center_x, center_y), r,  fill = False )

print(center_x, center_y)

f, ax = plt.subplots(1,2, figsize = (10,20))
ax[0].imshow(final_im)
ax[0].scatter(center_x, center_y, s=1)
ax[0].add_patch(circle1)
ax[1].imshow(blur)

In [ ]:
#@title 1.4. Testes com o centro obtido pela transformada de Hough


ai = pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=0.1, pixel1=1e-4, pixel2=1e-4)
ai.setFit2D(1024, center_x, center_y+2)
stacked1= masked_stacked
binning = 798
res1 = ai.integrate1d(stacked1, binning, unit='2th_deg', azimuth_range=(-50, -30), method = "ocl_lut_integr")
res2 = ai.integrate1d(stacked1, binning, unit='2th_deg', azimuth_range=(120, 140), method = "ocl_lut_integr")
res3 = ai.integrate1d(stacked1, binning, unit='2th_deg', azimuth_range=(80, 100), method = "ocl_lut_integr")
res4 = ai.integrate1d(stacked1, binning, unit='2th_deg', azimuth_range=(-100, -80), method = "ocl_lut_integr")

res1, res2 = np.array(res1), np.array(res2)
res3, res4 = np.array(res3), np.array(res4)

pico_x = np.argmax(res1[1][100:500]), np.argmax(res2[1][100:500])

erro_x = analysis.MSE(res1[1][100:500], res2[1][100:500])
erro_y = analysis.MSE(res3[1][100:500], res4[1][100:500])
print(erro_x, erro_y)

best_error = 1000000

if (erro_x+erro_y) < best_error:
  best_error = erro_x + erro_y
  best_centro = [center_x, center_y]

fig,ax= plt.subplots(2,2,figsize=(20, 10))

ax[0, 0].set_title('Integração em x')
ax[0, 0].plot(res1[1][0:int(0.33*binning)])
ax[0, 0].plot(res2[1][0:int(0.33*binning)])
#ax[0, 0].text(200, 0.5 * res3[1][100:600].max(), f'Melhor erro: {best_error:.2f} com centro 'f'{best_centro}', fontsize=12, color='red')
#ax[0, 0].text(200, 0.4 * res3[1][100:600].max(), f'EQM: {(erro_x + erro_y):.2f} com centro 'f'{center_x, center_y}', fontsize=12, color='red')

ax[0, 1].set_title('Integração em y')
#ax[0, 1].set_xlim(0, 500)
ax[0, 1].plot(res3[1][0:int(0.33*binning)])
ax[0, 1].plot(res4[1][0:int(0.33*binning)])

ax[1, 0].plot(res1[1])
ax[1, 0].plot(res2[1])

ax[1, 1].plot(res3[1])
ax[1, 1].plot(res4[1])

plt.tight_layout()
plt.show()


In [ ]:
#@title 1.5. Integrando e salvando

ai = pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=0.1,  pixel1=1e-4, pixel2=1e-4)
ai.setFit2D(1024, center_x, center_y+2)
res = ai.integrate1d(x, binning, unit='2th_deg', azimuth_range=(-180, 180), method = "ocl_lut_integr")


"""
ai.setFit2D(2048, center_x, center_y+2)
res1 = ai.integrate1d(binning, 3108, unit='2th_deg', azimuth_range=(-180, 180), method = "ocl_lut_integr")

center_x2, center_y2 = center_x, center_y+1
ai.setFit2D(2048, center_x2, center_y2)
res2 = ai.integrate1d(masked_stacked, 3000, unit='2th_deg', azimuth_range=(-180, 180), method = "ocl_lut_integr")

center_x3, center_y3 = center_x-1, center_y
ai.setFit2D(2048, center_x3, center_y3)
res3 = ai.integrate1d(masked_stacked, 3000, unit='2th_deg', azimuth_range=(-180, 180), method = "ocl_lut_integr")

center_x4, center_y4 = center_x, center_y-1
ai.setFit2D(2048, center_x4, center_y4)
res4 = ai.integrate1d(masked_stacked, 3000, unit='2th_deg', azimuth_range=(-180, 180), method = "ocl_lut_integr")
"""

#process.save_iq_only_y(res1, "Fe3O4_Oleyl-OH_ROI1")
process.save_iq_only_y(res, "Corning_CAS40_ROI1_0.0063s_20_B4")

plt.plot(res[1])
#plt.plot(res1[1])
#plt.plot(res2[1])
#plt.plot(res3[1])
#plt.plot(res4[1])
#plt.xlim([1000, 1600])
#plt.ylim([0, 100000])